## image classification

### import libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input, Lambda, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator 

c:\users\j7000\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\j7000\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\j7000\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\j7000\appd

### VGG16 model

In [6]:
IMAGE_SIZE=[128, 128]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [7]:
#dont train existing weight
for layer in vgg.layers:
    layer.trainable = False

In [12]:
folder = glob("Data/*")
#we need to add more layer
x = Flatten()(vgg.output)
prediction = Dense(len(folder), activation = tf.nn.softmax)(x)
model = Model(inputs = vgg.input, outputs = prediction)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [15]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

#### Data Augmentation

In [16]:
#import image from datasets using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  featurewise_center = False, #set input mean 0 over the data
                                  samplewise_center = False, #set each sample mean 0
                                  featurewise_std_normalization=False, #divide inputs by std of the datasets
                                  zca_whitening = False,
                                  rotation_range=10, #randomly rotate images in the range 0-100
                                  width_shift_range = 0.2, #randomly shift image vertically. fraction of total height.
                                  vertical_flip = False
                                  )

#test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
# Make sure I provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Data',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 361 images belonging to 3 classes.


In [19]:
model.fit_generator(
    training_set,
    epochs=10,
    steps_per_epoch = len(training_set)
)

Epoch 1/10
12/12 [==============================] - 183s 15s/step - loss: 0.6656 - acc: 0.7191
Epoch 2/10
12/12 [==============================] - 166s 14s/step - loss: 0.2990 - acc: 0.9032
Epoch 3/10
12/12 [==============================] - 166s 14s/step - loss: 0.2047 - acc: 0.9293
Epoch 4/10
12/12 [==============================] - 166s 14s/step - loss: 0.1711 - acc: 0.9424
Epoch 5/10
12/12 [==============================] - 165s 14s/step - loss: 0.1272 - acc: 0.9686
Epoch 6/10
12/12 [==============================] - 167s 14s/step - loss: 0.1024 - acc: 0.9791
Epoch 7/10
12/12 [==============================] - 166s 14s/step - loss: 0.0890 - acc: 0.9817
Epoch 8/10
12/12 [==============================] - 166s 14s/step - loss: 0.0871 - acc: 0.9869
Epoch 9/10
12/12 [==============================] - 166s 14s/step - loss: 0.0810 - acc: 0.9843
Epoch 10/10
12/12 [==============================] - 165s 14s/step - loss: 0.0630 - acc: 0.9921


In [20]:
model.save('vgg16_model.h5')